In [1]:
from bs4 import BeautifulSoup
import datetime
import requests
import pandas as pd
import numpy as np
import time

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_town_dict = {'ARI': 'Arizona',
                        'ATL': 'Atlanta',
                        'BAL': 'Baltimore',
                        'BUF': 'Buffalo',
                        'CAR': 'Carolina',
                        'CHI': 'Chicago',
                        'CIN': 'Cincinnati',
                        'CLE': 'Cleveland',
                        'DAL': 'Dallas',
                        'DEN': 'Denver',
                        'DET': 'Detroit',
                        'GB': 'Green Bay',
                        'HOU': 'Houston',
                        'IND': 'Indianapolis',
                        'JAX': 'Jacksonville',
                        'KC': 'Kansas City',
                        'LAC': 'L.A. Chargers',
                        'LAR': 'L.A. Rams',
                        'MIA': 'Miami',
                        'MIN': 'Minnesota',
                        'NE': 'New England',
                        'NO': 'New Orleans',
                        'NYG': 'N.Y. Giants',
                        'NYJ': 'N.Y. Jets',
                        'OAK': 'Oakland',
                        'PHI': 'Philadelphia',
                        'PIT': 'Pittsburgh',
                        'SEA': 'Seattle',
                        'SF': 'San Francisco',
                        'TB': 'Tampa Bay',
                        'TEN': 'Tennessee',
                        'WSH': 'Washington'}
town_to_acronym_dict = {'Arizona': 'ARI',
                        'Atlanta': 'ATL',
                        'Baltimore': 'BAL',
                        'Buffalo': 'BUF',
                        'Carolina': 'CAR',
                        'Chicago': 'CHI',
                        'Cincinnati': 'CIN',
                        'Cleveland': 'CLE',
                        'Dallas': 'DAL',
                        'Denver': 'DEN',
                        'Detroit': 'DET',
                        'Green Bay': 'GB',
                        'Houston': 'HOU',
                        'Indianapolis': 'IND',
                        'Jacksonville': 'JAX',
                        'Kansas City': 'KC',
                        'L.A. Chargers': 'LAC',
                        'L.A. Rams': 'LAR',
                        'Miami': 'MIA',
                        'Minnesota': 'MIN',
                        'New England': 'NE',
                        'New Orleans': 'NO',
                        'N.Y. Giants': 'NYG',
                        'N.Y. Jets': 'NYJ',
                        'Oakland': 'OAK',
                        'Philadelphia': 'PHI',
                        'Pittsburgh': 'PIT',
                        'Seattle': 'SEA',
                        'San Francisco': 'SF',
                        'Tampa Bay': 'TB',
                        'Tennessee': 'TEN',
                        'Washington': 'WSH'}
nickname_to_town_dict = {'Cardinals': 'Arizona',
                        'Falcons': 'Atlanta',
                        'Ravens': 'Baltimore',
                        'Bills': 'Buffalo',
                        'Panthers': 'Carolina',
                        'Bears': 'Chicago',
                        'Bengals': 'Cincinnati',
                        'Browns': 'Cleveland',
                        'Cowboys': 'Dallas',
                        'Broncos': 'Denver',
                        'Lions': 'Detroit',
                        'Packers': 'Green Bay',
                        'Texans': 'Houston',
                        'Colts': 'Indianapolis',
                        'Jaguars': 'Jacksonville',
                        'Chiefs': 'Kansas City',
                        'Chargers': 'L.A. Chargers',
                        'Rams': 'L.A. Rams',
                        'Dolphins': 'Miami',
                        'Vikings': 'Minnesota',
                        'Patriots': 'New England',
                        'Saints': 'New Orleans',
                        'Giants': 'N.Y. Giants',
                        'Jets': 'N.Y. Jets',
                        'Raiders': 'Oakland',
                        'Eagles': 'Philadelphia',
                        'Steelers': 'Pittsburgh',
                        'Seahawks': 'Seattle',
                        '49ers': 'San Francisco',
                        'Buccaneers': 'Tampa Bay',
                        'Titans': 'Tennessee',
                        'Redskins': 'Washington'}

In [34]:
def initial_538_predictions(week_num):
    """Creates dataframe with initial values from 538
    Args:
        week_num: week number of games
    Returns:
        dataframe of 538 predictions
        year: year of data
    """
    df_columns = ['week', 'date', 'team', '538 win%']
    df = pd.DataFrame(columns=df_columns)
    
    output_file = 'z:\python_projects\aaa.exe'
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.get_text()[5] == str(week_num):
            for date in week.findAll('div', attrs={'class': 'days'}):
                for weekday in date.findAll('div', attrs={'class': 'day'}):  
                    for game in weekday.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in weekday.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text().strip()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:] + ' ' + str(year)
                            date = datetime.datetime.strptime(date, '%b %d %Y')
                            year = date.year
                            month = date.month
                            day = date.day
                            date = date.strftime('%m/%d/%Y')

                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                squad = matchup.find('td', attrs={'class': 'td text team'})
                                win_percentage = matchup.find('td', attrs={'class': 'td number chance'}).get_text().strip()

                                if squad:
                                    team = squad.get_text().strip()
                                    result = ''
                                    
                                    if num_teams == 0:
                                        df = df.append(pd.Series([week_finder(datetime.date(year, month, day)),
                                                                  date, team, win_percentage], index=df.columns),
                                                                  ignore_index=True)
                                    else:
                                        df = df.append(pd.Series([week_finder(datetime.date(year, month, day)),
                                                                  '', team, win_percentage], index=df.columns),
                                                                  ignore_index=True)
    df.drop_duplicates(inplace=True)
    
    return df, year
    
def getting_spreadsheet(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
    
def separating_games(df):
    """Separates old games and new games
    Args:
        df: dataframe of data
    Returns:
        old_games: games with results already
        current_game: games without results
    """
    old_games = df[df['result'] != '']
    old_games_index = df[df['result'] != ''].index 
    current_games = df.drop(old_games_index)
    return old_games, current_games
    
def predictions_538(week_num):
    """Loads 538 predictions
    Args:
        week_num: week number of games
    Returns:
        df: dataframe with 538 predictions
        year: year of games. Needed for Vegas odds
    """
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    cols = ['week', 'date', 'team', '538 win%']
    blank_rows = ['']*(len(df.columns)-3)
    cols.extend(blank_rows)
    predictions = pd.DataFrame(columns=cols)

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.h3.get_text()[-2:].strip() == str(week_num):
            for date in week.findAll('div', attrs={'class': 'days'}):
                for weekday in date.findAll('div', attrs={'class': 'day'}):  
                    for game in weekday.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in weekday.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text().strip()
                            date = date.split(', ')[1]
                            date = date[0:3] + ' ' + date[-2:] + ' ' + str(year)
                            date = datetime.datetime.strptime(date, '%b %d %Y')
                            year = date.year
                            month = date.month
                            day = date.day
                            date = date.strftime('%m/%d/%Y')


                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                squad = matchup.find('td', attrs={'class': 'td text team'})
                                win_percentage = matchup.find('td', attrs={'class': 'td number chance'}).get_text().strip()

                                if squad:
                                    team = squad.get_text().strip()
                                    result = ''
                                    
                                    if num_teams == 0:
                                        row_data = [week_finder(datetime.date(year, month, day)), date, team, win_percentage]
                                        row_data.extend(blank_rows)
                                        
                                        predictions = predictions.append(pd.Series(row_data, index=predictions.columns), 
                                                                         ignore_index=True)
                                    else:
                                        row_data = [week_finder(datetime.date(year, month, day)), date, team, win_percentage]
                                        row_data.extend(blank_rows)

                                        predictions = predictions.append(pd.Series(row_data, index=predictions.columns), 
                                                                         ignore_index=True)
  
    return predictions, year
    
def predictions_fox(df):
    """Loads fox predictions
    Args:
    df: dataframe of data that will be merged with
    Returns:
        df: dataframe including Fox predictions
    """
    predictions_columns = ['week', 'team', 'Fox win%']
    predictions = pd.DataFrame(columns=predictions_columns)

    year = 2019
    result = ''
    URL = 'https://www.foxsports.com/nfl/predictions'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for week in soup.findAll('span', attrs={'class': 'wisbb_pageInfoSecondaryText'}):
        week_num = int(week.get_text().strip()[5:])


    for matchup in soup.findAll('div', attrs={'class': 'wisbb_predictionChip'}):
        for num, team_name in enumerate(matchup.findAll('span', attrs={'class': 'wisbb_teamName'})):
            if num == 0:
                away_team = team_name.get_text().strip()
            elif num == 1:
                home_team = team_name.get_text().strip()
            else:
                continue

        for num, fox_predictions in enumerate(matchup.findAll('span', attrs={'class': 'wisbb_predData'})):
            prediction_text = fox_predictions.get_text()
            team_acronym = fox_predictions.get_text()[0:3].strip()
            if team_acronym in acronym_to_town_dict and len(prediction_text)>4 and num ==1:
                if acronym_to_town_dict[team_acronym] == nickname_to_town_dict[away_team]:
                    away_win_percentage_int = int(round(float(prediction_text.split('(')[1][:-2])))
                    away_win_percentage_string = str(away_win_percentage_int) + '%'
                    home_win_percentage_int = 100 - away_win_percentage_int
                    home_win_percentage_string = str(home_win_percentage_int) + '%'

                elif acronym_to_town_dict[team_acronym] == nickname_to_town_dict[home_team]:
                    home_win_percentage_int = int(round(float(prediction_text.split('(')[1][:-2])))
                    home_win_percentage_string = str(home_win_percentage_int) + '%'
                    away_win_percentage_int = 100 - home_win_percentage_int
                    away_win_percentage_string = str(away_win_percentage_int) + '%'
                else:
                    continue

                predictions = predictions.append(pd.Series([week_num, nickname_to_town_dict[away_team], away_win_percentage_string],
                                             index=predictions.columns), ignore_index=True)
                predictions = predictions.append(pd.Series([week_num, nickname_to_town_dict[home_team], home_win_percentage_string],
                                             index=predictions.columns), ignore_index=True)

    df = pd.merge(predictions, df)
    
    return df
    
def loading_odds(df, year):
    """Loads odds to spreadsheet
    Args:
        df: dataframe that odds will be written to
        year: year that will be used to gather the dates of the games
    Returns:
        df: dataframe of datawith odds
    """
    names = ['Open', 'odds','Westgate','MGM Mirage', 'betMGM',
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    output_file = 'z:\python_projects\aaa.exe'
    temp_away_list = []
    temp_home_list = []
    teams_list = []
    dates_list = []
    URL = 'http://www.vegasinsider.com/nfl/odds/las-vegas/money/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
        for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
            for game_info in games.findAll('td'):  
                for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):
                    date = date.get_text().split()[0]
                    month = date.split('/')[0]
                    day = date.split('/')[1]

                    date = str(month) + '/' + str(day) + '/' + str(year)
                    dates_list.extend([date, date])

                for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}):
                    teams_list.append(team_name.get_text())

            for num, spread in enumerate(games.findAll('td',attrs={'class': 
                                                                  ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap',
                                                                   'viCellBg1 cellTextHot cellBorderL1 center_text nowrap',
                                                                   'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap',
                                                                   'viCellBg2 cellTextHot cellBorderL1 center_text nowrap']})):           
                spread_text = spread.get_text().strip()
                if spread_text == '' or spread_text == 'XXXX':
                    away_spread = np.nan
                    home_spread = np.nan
                elif(spread_text[4] == '+' or spread_text[4] == '-'):
                    away_spread = spread_text[0:4]
                    home_spread = spread_text[4:]
                else:
                    away_spread = spread_text[0:5]
                    home_spread = spread_text[5:]

                temp_away_list.append(away_spread)
                temp_home_list.append(home_spread)
                
                if len(temp_home_list) == 9:
                    book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                    book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                    temp_away_list = []
                    temp_home_list = []
                    
    book['team'] = teams_list
    book['date'] = dates_list
    book = book[book.date <= df.date.max()]    # Only getting odds for games in current week
    odds = book[['team', 'odds']]
    df = pd.merge(odds, df)[['week', 'date', 'team', '538 win%', 'Fox win%', 'odds']]

    return df
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('/')
        day = split_date[1]
        day = day.zfill(2)
        date = split_date[0] + day + split_date[2]
        date = datetime.datetime.strptime(date, '%m%d%Y')
        date = date.strftime('%m/%d/%Y')
        return(date)
        
def date_ffill_formatter(df):
    """Formats dates to be in every other row
    Args:
        df: df of data
    """
    for index, row in df.iterrows():
        if index%2 == 0:
            continue
        else:
            df.at[index, 'date'] = ''            
    
def combining_data(df_top, df_bottom):
    """Appends two dataframes
    Args:
        df_top: dataframe of data to go on top
        df_bottom: dataframe of data to go on bottom
    Returns:
        df: combined dataframe
    """
    cols = df_top.columns
    df = pd.concat([df_top, df_bottom], ignore_index=True, )
    df = df[cols]
    df.replace(np.nan, '', inplace=True)
    return(df)
    
def game_outcomes(game_dates):
    """Finds winners and losers of games
    Args:
        game_dates: date of game
    Returns:
        winners: dict with dates as keys and a list of winners as values
        losers: dict with dates as keys and a list of losers ass values
    """
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    winner_list = []
    loser_list = []
    winners = {}
    losers = {}

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        for week_text in week.findAll('h3', attrs={'class': 'h3'}):
            week_num_text = week_text.get_text()[5:]
            
        
        for date in week.findAll('div', attrs={'class': 'days'}):
            for day in date.findAll('div', attrs={'class': 'day'}):  

                # Finding date of each game  
                for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                    date = h4.get_text()
                    date = date.split(', ')[1]
                    date = date[0:3] + ' ' + date[-2:].strip() + ' ' + str(year)
                    date = datetime.datetime.strptime(date, '%b %d %Y')
                    date = date.strftime('%m/%d/%Y')

                    if date in game_dates:
                        # Finding data for each game
                        for game_body in day.findAll('table', attrs={'class': 'game-body'}):
                            for winner_tag in game_body.findAll('td', attrs={'class': 'td text team winner'}):
                                winner = winner_tag.get_text().strip()
                               
                                winner_list.append(winner)
                                
                            for loser_tag in game_body.findAll('td', attrs={'class': 'td text team loser'}):
                                loser = loser_tag.get_text().strip()
                                
                                loser_list.append(loser)
        
                        winners[date] = winner_list
                        losers[date] = loser_list
                        winner_list = []
                        loser_list = []

    return winners, losers
def implied_probability(row):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """
    if 'implied' in row.index:
        return row['implied']
    elif row['odds'] == '':
        return('')
    elif row['odds'][0] == '+':
        return(round(100.0/(100+int(row['odds'][1:])), 2))
    elif row['odds'][0] == '-':
        return(round(int(row['odds'][1:])/(100.0+int(row['odds'][1:])), 2))
    else:
        return('')    
    
    
def pick(row, predictor):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
        predictor: name of predicting column
    Returns:
        pick if there is one
    """
    if predictor + ' pick' in row.index:
        if row[predictor + ' pick'] == '':
            return ''
        else:    
            return(town_to_acronym_dict[row[predictor + ' pick']])
    elif row['implied'] == '':
        return('')
    elif (float(row[predictor + ' win%'][:-1])/100.0 > row['implied']):
        return(town_to_acronym_dict[row.team])
    else:
        return('')
    
def spreadsheet_formatter(df):
    """Formats spreadsheet
    Args:
        df: dataframe of data
    """
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
        
    df.replace(np.nan, '', inplace=True)
    
    
def odds_formatter(row):
    """Formats odds
    Args:
        row: row of data from dataframe
    Returns:
        formatted odds
    """
    if row['odds'] == '':
        return(row['odds'])
    elif str(row['odds'])[0] == '+' or str(row['odds'])[0] == '-':
        return(str(row['odds']))
    elif int(row['odds']) >= 100:
        return('+' + str(row['odds']))
    if int(row['odds']) <= -100:
        return(str(row['odds']))
    else:
        return(str(row['odds'])) 
    
def team_won_lost(row, winners, loser):
    """Determines team outcomes
    Args:
        row: row of data from dataframe
        winners: dict with dates as keys and a list of winners as values
        losers: dict with dates as keys and a list of losers as values
    Returns:
        updated entry for winner/loser
    """
    if row.date != '':
        if row.date in winners.keys():
            if row.team in winners[row.date]:
                return('w')
            elif row.team in losers[row.date]:
                return('l')
            else:
                return('')
        else:
            return('')
    else:
        return('')
    
def money_won_lost(row, predictor):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
        predictor: name of predictor
    Returns:
        amount won/lost
    """
    if row[predictor + ' pick'] == '':
        return(int(0))
    else:
        if row['result'] == 'w':
            if row['odds'][0] == '+':
                return(int(row['odds'][1:]))
            else:
                return(100)
        elif row['result'] == 'l':
            if row['odds'][0] == '+':
                return(-100)
            else:
                return(int(row['odds']))
        else:
            return(int(0))
                
def week_finder(date):
    """Finds what week games are in
    Args:
        date: date in mm/dd/yyyy form
    Returns:
        week: week of games
    """
    if date < datetime.date(2019, 9, 10):
        week = 1
    elif date < datetime.date(2019, 9, 17):
        week = 2
    elif date < datetime.date(2019, 9, 24):
        week = 3
    elif date < datetime.date(2019, 10, 1):
        week = 4
    elif date < datetime.date(2019, 10, 8):
        week = 5
    elif date < datetime.date(2019, 10, 15):
        week = 6
    elif date < datetime.date(2019, 10, 22):
        week = 7
    elif date < datetime.date(2019, 10, 29):
        week = 8
    elif date < datetime.date(2019, 11, 5):
        week = 9
    elif date < datetime.date(2019, 11, 12):
        week = 10
    elif date < datetime.date(2019, 11, 19):
        week = 11
    elif date < datetime.date(2019, 11, 26):
        week = 12
    elif date < datetime.date(2019, 12, 3):
        week = 13
    elif date < datetime.date(2019, 12, 10):
        week = 14
    elif date < datetime.date(2019, 12, 17):
        week = 15
    elif date < datetime.date(2019, 12, 24):
        week = 16
    elif date < datetime.date(2019, 12, 31):
        week = 17
    else:
        week = 0
        
    return(week)
    
def writing_spreadsheet(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)

In [35]:
file_name = 'z:\\python projects\\NFL Game Outcome Spreadsheet.csv'

df = getting_spreadsheet(file_name) # Retrieving Spreadsheet
spreadsheet_formatter(df)
old_games, current_games = separating_games(df)

if not current_games.empty:
    current_games['date'] = current_games['date'].replace('', np.nan).ffill()
    current_games['odds'] = current_games.apply(lambda row: odds_formatter(row), axis=1)
    winners, losers = game_outcomes(current_games.date.unique())
    current_games['result'] = current_games.apply(lambda row: team_won_lost(row, winners, losers), axis=1)
    current_games['538 w/l'] = current_games.apply(lambda row: money_won_lost(row, '538'), axis=1)
    current_games['Fox w/l'] = current_games.apply(lambda row: money_won_lost(row, 'Fox'), axis=1)
    df = combining_data(old_games, current_games)
print(df)
df['odds'] = df['odds'] = current_games.apply(lambda row: odds_formatter(row), axis=1) 
df['date'] = df.apply(lambda row: date_formatter(row), axis=1)

if datetime.date.today().strftime('%m/%d/%Y') > df.date.max():
    new_games, year = predictions_538(week_num=week_finder(datetime.date.today()))
    new_games = predictions_fox(new_games)
    new_games = loading_odds(new_games, year)
    new_games['implied'] = new_games.apply(lambda row: implied_probability(row), axis=1)
    new_games['538 pick'] = new_games.apply(lambda row: pick(row, '538'), axis=1)
    new_games['Fox pick'] = new_games.apply(lambda row: pick(row, 'Fox'), axis=1)
    new_games['538 w/l'] = 0
    new_games['Fox w/l'] = 0
    df = combining_data(df, new_games)

df['538 w/l'] = df['538 w/l'].astype(int)
df['Fox w/l'] = df['Fox w/l'].astype(int)
df['538 total'] = int(df.loc[df['538 w/l'] != np.nan]['538 w/l'].sum())
df['Fox total'] = int(df.loc[df['Fox w/l'] != np.nan]['Fox w/l'].sum())
date_ffill_formatter(df)
df
#writing_spreadsheet(df, file_name)

115
230
140
160
165
260
     week        date           team 538 win% Fox win%  odds  implied  \
0       1  09/05/2019      Green Bay      36%      44%   170     0.37   
1       1                    Chicago      64%      56%  -200     0.67   
2       1  09/08/2019      Tennessee      40%      49%   205     0.33   
3       1                  Cleveland      60%      51%  -245     0.71   
4       1  09/08/2019      Baltimore      61%      72%  -235     0.70   
5       1                      Miami      39%      28%   195     0.34   
6       1  09/08/2019        Buffalo      45%      49%   130     0.43   
7       1                  N.Y. Jets      55%      51%  -150     0.60   
8       1  09/08/2019     Washington      23%      36%   400     0.20   
9       1               Philadelphia      77%      64%  -500     0.83   
10      1  09/08/2019    Kansas City      58%      69%  -175     0.64   
11      1               Jacksonville      42%      31%   155     0.39   
12      1  09/08/2019      

,week,date,team,538 win%,Fox win%,odds,implied,538 pick,Fox pick,result,538 w/l,Fox w/l,538 total,Fox total
0,1,09/05/2019,Green Bay,36%,44%,NaN,0.37,,GB,w,0,170,-1580,535
1,1,,Chicago,64%,56%,NaN,0.67,,,l,0,0,-1580,535
2,1,09/08/2019,Tennessee,40%,49%,NaN,0.33,TEN,TEN,w,0,205,-1580,535
3,1,,Cleveland,60%,51%,NaN,0.71,,,l,0,0,-1580,535
4,1,09/08/2019,Baltimore,61%,72%,NaN,0.70,,BAL,w,0,100,-1580,535
5,1,,Miami,39%,28%,NaN,0.34,MIA,,l,-100,0,-1580,535
6,1,09/08/2019,Buffalo,45%,49%,NaN,0.43,BUF,BUF,w,130,130,-1580,535
7,1,,N.Y. Jets,55%,51%,NaN,0.60,,,l,0,0,-1580,535
8,1,09/08/2019,Washington,23%,36%,NaN,0.20,WSH,WSH,l,-100,-100,-1580,535
9,1,,Philadelphia,77%,64%,NaN,0.83,,,w,0,0,-1580,535
